# Підготовка даних

## Файл csv із збігами та їх прізвищами

Мають бути такі колонки

- surname - прізвище предка збіженця. У чоловічому роді, українською мовою
- name - ім'я збіженця
- cm - число, спільна днк в сантиморганах
- url - покликання на збіженця

# Параметри

In [3]:
# Всі необхідні файли потрібно покласи у цей каталог
directory = "data"

# Файл із збіженцями та прізвищами їх предків
filename = "DNA Genealogy - Surnames_my"

multy = 1000

# Відкриваємо вхідний файл із збіженцями прізвищами їх предків та базу даних

In [4]:
import sqlite3

db = sqlite3.connect(f'{directory}/surnames.sqlite3')

In [3]:
import pandas as pd
matches_df = pd.read_csv(f'{directory}/{filename}.csv')
matches_df

,surname,group,name,cm,url
0,Шиліпук,NaN,Olya Shylipuk,41.7,https://www.myheritage.com.ua/dna/match/D-FE4A...
1,Бурчак,NaN,Olya Shylipuk,41.7,https://www.myheritage.com.ua/dna/match/D-FE4A...
2,Шмаленко,NaN,Inna Shmalenko,40.5,https://www.myheritage.com.ua/dna/match/D-FE4A...
3,Купчик,NaN,Inna Shmalenko,40.5,https://www.myheritage.com.ua/dna/match/D-FE4A...
4,Полікарпенко,NaN,Александр Поликаренко,37.7,https://www.myheritage.com.ua/dna/match/D-FE4A...
...,...,...,...,...,...
172,Філіпович,NaN,Олена Тихан,21.4,https://www.myheritage.com.ua/dna/match/D-FE4A...
173,Гетун,NaN,Настя Шкинь,21.4,https://www.myheritage.com.ua/dna/match/D-FE4A...
174,Шкінь,NaN,Настя Шкинь,21.4,https://www.myheritage.com.ua/dna/match/D-FE4A...
175,Тумаш,NaN,Альона Хвалько,21.3,https://www.myheritage.com.ua/dna/match/D-FE4A...


# Підготовка таблиць

In [5]:
table_distribution = "distribution"
table_raw = "raw"

cursor = db.cursor()


In [5]:
cursor.execute(f'DROP TABLE IF EXISTS {table_distribution}')
cursor.execute(
    f'''
    CREATE TABLE {table_distribution} (
    	"region"	INTEGER NOT NULL,
    	"county"	INTEGER NOT NULL,
    	"city"	TEXT NOT NULL,
    	"score"	REAL NOT NULL,
    	PRIMARY KEY("region","county","city")
    )
    '''
)

In [6]:
cursor.execute(f'DROP TABLE IF EXISTS {table_raw}')
cursor.execute(
    f'''
    CREATE TABLE "{table_raw}" (
    	"surname"	INTEGER NOT NULL,
    	"city"	TEXT NOT NULL,
    	"county"	INTEGER NOT NULL,
    	"region"	INTEGER NOT NULL,
    	"cnt"	INTEGER NOT NULL,
    	"match"	INTEGER NOT NULL,
    	"url"	TEXT NOT NULL
    );
    '''
)

In [7]:
db.commit()

# Групування збіженців

In [8]:
match_grouped = matches_df.groupby(["url", "cm", "name"]).surname

In [9]:
for (url, cm, match), surnames in match_grouped:
    print(match)
    cm_float = float(cm)
    str_surnames = "','".join(map(lambda t: t.upper(), surnames))
    select = f'''
        SELECT 
            COUNT(*) AS cn,
        	"Областьнародження" as region,
        	IFNULL("Районнародження",0) as county,
        	"Названаселеногопунктународження(Україна)" as city,
            "Прізвищє" as surname
        FROM people 
        where 
        	"Прізвищє" in ('{str_surnames}') 
        	AND "Стать"=1 
        	AND "Названаселеногопунктународження(Україна)" IS NOT NULL 
        	AND "Областьнародження" IS NOT NULL
        GROUP BY region,
        	county,
        	city,
            surname
    '''

    def rec(row):
        (cnt, region, county, city, surname) = row
        return {
            'surname': surname,
            "region": region,
            "county": county,
            "city": city,
            "cnt": cnt
        }

    cursor.execute(select)
    res = list(map(rec, cursor.fetchall()))
    total = 0
    for t in res:
        # print(t)
        total = total + t["cnt"]

    def update_params(r):
        score = (cm_float*r["cnt"]*multy)/total
        return (r["region"], r["county"], r["city"], score, score)

    def insert_raw_params(r):
        return (r["surname"], r["city"], r["county"], r["region"], r["cnt"], match, url)

    cursor.executemany(
        f'''
        INSERT INTO {table_distribution}(region, county, city, score)
          VALUES(?, ?, ?, ?)
          ON CONFLICT(region, county, city) DO UPDATE SET
            score=score+?;
        ''',
        list(map(update_params, res))
    )

    cursor.executemany(
        f'''
        INSERT INTO {table_raw}(surname, city, county, region, cnt, match, url)
          VALUES(?, ?, ?, ?, ?, ?, ?)
        ''',
        list(map(insert_raw_params, res))
    )
    
    db.commit()
    # break

print("All records processed")
    

Ганна Лиштван
Анатолій Денисенко
Марія Михайлюк
John Michalchuk
Садовий Данило
Ser Roshko
Олена Тихан
Олеся Мазуркова
Tatiana Slatvitskaya
Olena Grybyk
Vadym Kovadlo
Alina Knysh
Natalia Kardash
Vladimir Oleshchuk
Pavlo Yelizarov
Ekaterina Buhl
Lukasz Nikoniuk
Olya Shylipuk
Андрей Ваврыш
Kelly Marceniuk
Alla Mykolaienko
Irina Jermolina
Alessandra Frabizzio
Svetlana Kletsova
Aleksy Krawczenko
Oxana Shevel
Julija Lemeshenko
Aliaksandra Turchak
Сергей Борисович Обломский
Maksym Chornopolsky
Paul Donald Bodnarchuk
Мирослава Стольницька
Anna Tovstukha
Mikhail Dubin
Юра Федорчук
Mykhalo Balaban
Альона Хвалько
Olga Saleh
Svetlana Berg
Nicole Laszuk
Яніна Брензей
Nicola C
Tetiana Shvorobei
Сергій Дмитрович Хватков
Тетяна Челяда
Oksana Baškė
Artiom Zalizko
Vira Kozyarovska
Александр Поликаренко
Ivan Ilnytskyi
Vitali Likhadzed
Мар'яна Булах
Любов Шальнева
Любов Шальнева
Natalia Zhyronkina
Оксана Побережник
Zhanna Samaruk
Алексей Ципинюк
Inna Shmalenko
Kozový Daniel
Volodimir Nezheria
Uladzislau L

# Створимо с

In [1]:
import xlsxwriter

In [6]:
import os, errno

resfile = f'{directory}/{filename}.xlsx'
try:
    os.remove(filename)
except OSError:
    pass
workbook = xlsxwriter.Workbook(resfile)

In [7]:
def add_row(ws, rn, content, format = None):
    for i, c in enumerate(content):
        # print(f"Write {c} to ({i},{rn})")
        ws.write(rn, i, c, format)

In [15]:
header_format = workbook.add_format()
header_format.set_bold()

def add_sheet_sql(name, header, sql, width = None):
    worksheet = workbook.add_worksheet(name)
    # if width is not None:
    #     for w in width:
            
    add_row(worksheet, 0, header, header_format)
    cursor.execute(sql)
    for i, row in enumerate(cursor.fetchall()):
        add_row(worksheet, i+1, row)

    worksheet.autofit()

In [17]:
add_sheet_sql("Топ 100", ["Область", "Район", "НП", "Ранг"], 
              '''
            SELECT
            	IFNULL(r.name, d.region) AS region,
            	IFNULL(c.name, d.county) as county,
            	city,
            	d.score
            FROM	
            	(SELECT county, sum(score) AS score
            	FROM vw_distribution
            	GROUP BY county) g
            INNER JOIN vw_distribution d ON g.county = d.county
            INNER JOIN regions r ON d.region=r.id
            LEFT JOIN counties c ON d.county=c.id
            ORDER BY g.score DESC, d.score DESC          
              '''
             )

In [18]:
add_sheet_sql("Райони", ["Область", "Район", "Ранг"], 
              '''
            SELECT
                IFNULL(r.name, d.region) as region,
                IFNULL(c.name, d.county) as county,
                d.score
            FROM
            (SELECT county, region, sum(score) as score
            FROM distribution d
            WHERE county > 0
            GROUP BY county, region
            ORDER BY sum(score) DESC) as d
            LEFT JOIN counties as c ON d.county = c.id
            LEFT JOIN regions as r on d.region = r.id   
              '''
             )

In [19]:
add_sheet_sql("Області", ["Область", "Ранг"], 
              '''
            SELECT
                IFNULL(r.name, d.region) as region,
                d.score
            FROM
            (SELECT region, sum(score) as score
            FROM distribution d
            GROUP BY region
            ORDER BY sum(score) DESC) as d
            LEFT JOIN regions as r on d.region = r.id
              '''
             )

In [20]:
add_sheet_sql("Всі Села", ["Область", "Район", "НП", "Ранг"], 
              '''
            SELECT
                IFNULL(r.name, d.region) AS region,
                IFNULL(c.name, d.county) as county,
                city,
                score
            FROM distribution d
            LEFT JOIN regions r ON d.region=r.id
            LEFT JOIN counties c ON d.county=c.id
            WHERE county > 0
            ORDER by score DESC        
              '''
             )

In [21]:
add_sheet_sql("Прізвища", ["Прізвище", "Область", "Район", "НП", "Кількість", "Збіг", "url"], 
              '''
            SELECT 
                surname,
                IFNULL(r.name, d.region) as region,
                IFNULL(c.name, d.county) as county,
                d.city,
                cnt,
                match,
                url
            FROM raw d
            LEFT JOIN counties as c ON d.county = c.id
            LEFT JOIN regions as r on d.region = r.id
            order by d.surname    
              '''
             )

In [22]:
workbook.close()

# Закрити базу даних

In [23]:
db.close()